In [2]:
library(tidyverse)

data <- read.csv("../data/combined_data.csv")



In [9]:
standardized_data <- data %>% select(-GEOID, -NAME)
standardized_data <- scale(standardized_data)



In [10]:
head(standardized_data)

has_computer,no_computer,with_internet,no_internet,total_population,work_from_home,X65_and_older,median_income,number_providers,mean_lowest_cost,mean_d_mbps,mean_u_mbps,mean_lat_ms
0.50245011,0.1993596,0.5167430,0.3205972,0.41151922,0.1292741,2.23493253,0.07418486,1.7094218,-0.18458632,-1.05913969,0.2831385,1.17991288
0.88153598,0.5037000,0.9194434,0.7032231,0.72118268,2.9303783,2.28240921,0.07416551,2.2108891,0.41923513,-0.64056563,-0.4304467,-0.15416598
-0.19922325,-0.2191085,-0.2714809,0.4088955,-0.07753548,-1.2814270,-0.74343819,0.07345717,1.2079545,0.39687137,-0.31447576,-0.2727836,0.10962741
-0.81523778,-0.6248957,-0.9413759,0.2911645,-0.87198242,-1.0480017,0.05100501,0.07347137,-0.2964475,1.12369349,-0.85523159,-0.4058615,-0.27548794
-0.07893639,0.9982532,-0.1913226,1.5077184,-0.16011240,-1.0784484,0.01302366,0.07354973,-0.2964475,1.12369349,-0.08992031,-0.4758288,-0.30026584
0.28010167,-0.4854063,0.1789328,0.5168156,0.10826259,-0.1751938,0.88026444,0.07377917,0.2050198,0.07383932,-0.57834859,-0.4065058,-0.07734433


### assignments

In [11]:
pc_data <- prcomp(standardized_data, scale = FALSE)

eigen_value <- pc_data$sdev^2

# invert the loadings
pc_data$rotation <- -pc_data$rotation

loadings <- pc_data$rotation[, 1:4]

pc_tract <- pc_data$x[, 1:4]

In [17]:
percent_explained <- eigen_value / sum(eigen_value)

In [21]:
as.list(percent_explained)

[[1]]
[1] 0.3122299

[[2]]
[1] 0.2035046

[[3]]
[1] 0.1237896

[[4]]
[1] 0.08766158

[[5]]
[1] 0.07540031

[[6]]
[1] 0.06344187

[[7]]
[1] 0.04212997

[[8]]
[1] 0.03538159

[[9]]
[1] 0.02299746

[[10]]
[1] 0.01296387

[[11]]
[1] 0.01163309

[[12]]
[1] 0.008580867

[[13]]
[1] 0.0002852431

In [23]:
sum(percent_explained[1:4])
sum(percent_explained[1:5])
sum(percent_explained[1:6])

[1] 0.7271857

[1] 0.8025861

[1] 0.8660279

### eigen value

In [13]:
as.list(eigen_value)

[[1]]
[1] 4.058989

[[2]]
[1] 2.64556

[[3]]
[1] 1.609265

[[4]]
[1] 1.139601

[[5]]
[1] 0.980204

[[6]]
[1] 0.8247443

[[7]]
[1] 0.5476896

[[8]]
[1] 0.4599606

[[9]]
[1] 0.298967

[[10]]
[1] 0.1685303

[[11]]
[1] 0.1512301

[[12]]
[1] 0.1115513

[[13]]
[1] 0.003708161

### loadings

In [25]:
# remove scientific notation
options(scipen=999)

loadings %>% as.data.frame()

,PC1,PC2,PC3,PC4
,<dbl>,<dbl>,<dbl>,<dbl>
has_computer,0.47288891,-0.02442760,0.10721627,-0.06093078
no_computer,0.20056267,-0.25288156,-0.58210439,-0.01030141
with_internet,0.47126532,0.00816404,0.13830333,-0.04908435
no_internet,0.20420996,-0.34706634,-0.49301848,0.01948705
total_population,0.44033591,-0.01913394,0.16992269,-0.04956443
work_from_home,0.23488701,-0.05887400,0.49066769,0.04533648
X65_and_older,0.35213674,-0.27837658,0.02454598,0.17397047
median_income,0.07656207,-0.10648033,-0.00130413,-0.09720139
number_providers,-0.10602038,-0.33992194,0.18558593,-0.51162516


### remove variables that don't exceed threshold of sqrt(1/27) = |.19245| based on loadings 

In [ ]:
as.matrix(loadings)

In [26]:
threshold = sqrt(1/13)
print(threshold)

[1] 0.2773501


In [27]:
loadings <- loadings %>%
    as.data.frame() %>% 
    filter(!(row.names(.) %in% c('median_income')))

loadings

,PC1,PC2,PC3,PC4
,<dbl>,<dbl>,<dbl>,<dbl>
has_computer,0.47288891,-0.02442760,0.10721627,-0.06093078
no_computer,0.20056267,-0.25288156,-0.58210439,-0.01030141
with_internet,0.47126532,0.00816404,0.13830333,-0.04908435
no_internet,0.20420996,-0.34706634,-0.49301848,0.01948705
total_population,0.44033591,-0.01913394,0.16992269,-0.04956443
work_from_home,0.23488701,-0.05887400,0.49066769,0.04533648
X65_and_older,0.35213674,-0.27837658,0.02454598,0.17397047
number_providers,-0.10602038,-0.33992194,0.18558593,-0.51162516
mean_lowest_cost,0.07707167,0.21997127,-0.01010764,0.58843052


### principal component 1 - 4

In [28]:
head(pc_tract)

PC1,PC2,PC3,PC4
-1.0278534,2.43313544,-0.7421958,0.6227943
-2.5567363,2.23886577,-1.7197546,0.2102354
0.9808996,0.41833790,0.4697185,0.3485766
1.5567772,-0.07165592,0.6048426,-1.1854047
-0.1631020,0.41428048,1.9776299,-1.0461623
-0.3830182,0.72454258,-0.1934017,-0.3277980


In [33]:
loadings[1,]

,PC1,PC2,PC3,PC4
,<dbl>,<dbl>,<dbl>,<dbl>
has_computer,0.4728889,-0.0244276,0.1072163,-0.06093078


multiply eigen vectors by eigen values

In [34]:
variable_weights <- loadings

for(i in 1:ncol(variable_weights)){
    variable_weights[i, ] <- variable_weights[i, ] * eigen_value[i]
}

prove that it worked

In [35]:
variable_weights %>% 
    as.data.frame() %>% 
    mutate(weight = PC1 + PC2 + PC3 + PC4)

,PC1,PC2,PC3,PC4,weight
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
has_computer,1.91945086,-0.09915138,0.43518968,-0.24731736,2.0081718
no_computer,0.53060062,-0.66901340,-1.53999223,-0.02725299,-1.7056580
with_internet,0.75839073,0.01313810,0.22256670,-0.07898973,0.9151058
no_internet,0.23271779,-0.39551701,-0.56184415,0.02220745,-0.7024359
total_population,0.44033591,-0.01913394,0.16992269,-0.04956443,0.5415602
work_from_home,0.23488701,-0.05887400,0.49066769,0.04533648,0.7120172
X65_and_older,0.35213674,-0.27837658,0.02454598,0.17397047,0.2722766
number_providers,-0.10602038,-0.33992194,0.18558593,-0.51162516,-0.7719815
mean_lowest_cost,0.07707167,0.21997127,-0.01010764,0.58843052,0.8753658


transmute instead of mutate and assign to variable

In [36]:
variable_weights <- variable_weights %>% 
    as.data.frame() %>% 
    transmute(weight = PC1 + PC2 + PC3 + PC4)

In [37]:
variable_weights

,weight
,<dbl>
has_computer,2.0081718
no_computer,-1.7056580
with_internet,0.9151058
no_internet,-0.7024359
total_population,0.5415602
work_from_home,0.7120172
X65_and_older,0.2722766
number_providers,-0.7719815
mean_lowest_cost,0.8753658


### separate tract and county from dataset to append later

In [30]:
geoid_tract <- standardized_data_all %>%
    select(GEOID, tract)

head(geoid_tract)

,GEOID,tract
,<dbl>,<dbl>
1,53001950100,9501.00
2,53001950200,9502.00
3,53001950301,9503.01
4,53001950302,9503.02
5,53001950303,9503.03
6,53001950400,9504.00


In [31]:
filtered_standardized_data <- standardized_data_all %>% 
    select(-GEOID, -tract, -dial_up)

head(filtered_standardized_data)

,not_citizen,naturalized_citizen,native_citizen,foreign_born,work_from_home,desktop_or_laptop,desktop_or_laptop_only,smartphone,smartphone_only,tablet_or_portable,⋯,access_with_no_subscription,no_internet_access,sixty_five_and_older,median_income,mean_income,mean_d_mbps,mean_u_mbps,mean_lat_ms,number_providers,mean_lowest_cost
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,-0.48236366,-0.1360753,-0.7772324,-0.4237144,-0.5101084,-0.8806845,0.0783680,-0.734196,0.5470533,-0.7648065,⋯,0.1866582,-0.1307050,-0.3984409,-0.37323672,-0.50677473,-1.621869,-0.93495618,0.7333922,1.7094218,-1.2748195
2,0.08777329,-1.0671850,-1.4235744,-0.3478686,-0.8653209,-1.4833040,-1.1695153,-1.434819,-0.4632060,-1.4361860,⋯,-0.6177302,-0.3367343,-1.2751771,-0.04914394,-0.60839056,-1.629913,-0.95350094,1.6330766,1.7094218,-0.7418166
3,2.02843174,-0.8181673,-1.8432087,1.2406807,-1.3118738,-1.8889890,-1.2440157,-1.823626,-0.3681227,-1.7682401,⋯,-0.5803168,0.1243789,-1.1770586,-0.93556901,-0.91747523,-1.321690,-0.79459603,1.5968650,-0.2964475,-1.6382306
4,2.49440905,0.9466105,-1.3890427,2.2856680,-1.0784484,-1.7649203,-1.5606428,-1.336655,0.2974598,-1.3149983,⋯,-0.4493698,-0.6114401,-1.1802237,-0.09364729,-0.26189039,-1.334392,-0.65140611,0.8212335,-0.2964475,-1.6382306
5,2.35187481,0.2320379,-1.2599245,1.8980114,-0.2259384,-1.5857099,-1.4488921,-1.284685,-0.1185293,-1.4895085,⋯,-0.6177302,-0.8272803,-1.1390772,-0.44758663,-0.09157226,-1.486311,-0.75776304,0.4994248,-0.2964475,-1.6382306
6,3.36057864,0.3727870,-0.9348768,2.7281021,-1.1088952,-1.4695186,-0.7038872,-1.173048,0.9511569,-1.1889632,⋯,-0.3558363,0.1734335,-1.0219681,0.12247378,-0.28001176,0.800291,-0.05116272,-0.1930008,0.2050198,-1.8643530


### multiply standardized values by variable weights

In [32]:
index <- filtered_standardized_data

for(i in 1:ncol(index)){
    index[, i] <- index[, i] * variable_weights[i,]
}

head(index)

,not_citizen,naturalized_citizen,native_citizen,foreign_born,work_from_home,desktop_or_laptop,desktop_or_laptop_only,smartphone,smartphone_only,tablet_or_portable,⋯,access_with_no_subscription,no_internet_access,sixty_five_and_older,median_income,mean_income,mean_d_mbps,mean_u_mbps,mean_lat_ms,number_providers,mean_lowest_cost
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,-0.42087952,-0.01605437,2.214360,-0.3036202,0.2977319,2.490386,-0.2000298,2.137220,-1.2675678,1.978232,⋯,-0.09056049,0.3983849,1.224534,-0.25411102,-0.33113546,-0.4275794,-0.32907114,-0.34962709,-0.11431958,1.6704821
2,0.07658533,-0.12590818,4.055810,-0.2492715,0.5050567,4.194463,2.9851202,4.176710,1.0732867,3.714807,⋯,0.29970266,1.0263558,3.919020,-0.03345870,-0.39753302,-0.4297001,-0.33559824,-0.77852996,-0.11431958,0.9720524
3,1.76987916,-0.09652867,5.251362,0.8890320,0.7656935,5.341653,3.1752784,5.308515,0.8529709,4.573691,⋯,0.28155089,-0.3791029,3.617471,-0.63696411,-0.59949434,-0.3484420,-0.27966939,-0.76126694,0.01982528,2.1466842
4,2.17646102,0.11168261,3.957428,1.6378364,0.6294515,4.990813,3.9834508,3.890957,-0.6892391,3.401346,⋯,0.21801968,1.8636503,3.627198,-0.06375795,-0.17112376,-0.3517908,-0.22927166,-0.39150331,0.01982528,2.1466842
5,2.05209480,0.02737620,3.589566,1.3600541,0.1318721,4.484045,3.6982135,3.739677,0.2746422,3.852730,⋯,0.29970266,2.5215246,3.500742,-0.30473072,-0.05983491,-0.3918417,-0.26670549,-0.23808876,0.01982528,2.1466842
6,2.93222493,0.04398201,2.663494,1.9548705,0.6472222,4.155481,1.7966314,3.414703,-2.2039096,3.075346,⋯,0.17264024,-0.5286198,3.140829,0.08338391,-0.18296457,0.2109837,-0.01800745,0.09200849,-0.01371094,2.4429877


### sum the rows

In [33]:
index <- index %>% 
    transmute(index = rowSums(.))

### append dataframes together

In [34]:
pc_index <- bind_cols(geoid_tract, index)

head(pc_index)

,GEOID,tract,index
,<dbl>,<dbl>,<dbl>
1,53001950100,9501.00,16.30880
2,53001950200,9502.00,38.48899
3,53001950301,9503.01,46.05822
4,53001950302,9503.02,47.42150
5,53001950303,9503.03,46.38311
6,53001950400,9504.00,35.05856


In [41]:
# write.csv(pc_index, "../data/pc_index.csv")